In [45]:
from keras import Input, layers, backend, Model, losses, datasets, models, metrics, optimizers, initializers
from keras.regularizers import l2
from keras.utils import Sequence
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import math

#initialise random generator
rng = np.random.default_rng()

In [46]:

class FelixSequence(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        """Here self.x is a list of paths to .npy input files. self.y is a
        corresponding list of paths to .npy output files."""
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        #print(np.array([np.load(file_name) for file_name in batch_x]).shape, np.array([np.load(file_name) for file_name in batch_y]).shape)
        return np.array([np.reshape(np.load(file_name), (128, 128, 1)) for file_name in batch_x]), np.array([np.reshape(np.load(file_name), (128, 128, 1)) for file_name in batch_y])
    

def gen_paths_labels(base_path = "D:\\Uni Work\\Masters Project\\electron_dists\\Data\\VAE_000_1\\Data"):
    """A generator to yield (data-paths, corresponding labels) tuples for each
    segment of data (typically training, validation, and testing)."""
    for segment in sorted(os.listdir(base_path)):
        segment_path = os.path.join(base_path, segment)
        input_paths = []
        output_paths = []
        for crystal in sorted(os.listdir(segment_path)):
            crystal_path = os.path.join(segment_path, crystal)
            files = sorted(os.listdir(crystal_path))
            input_paths.append(os.path.join(crystal_path, files[0]))
            output_paths.append(os.path.join(crystal_path, files[1]))
        yield [input_paths, output_paths]

def gen_paths_fromfile(Path):
    Paths = []
    with open(Path) as textFile:
        lines = [line.split() for line in textFile]
    for i in lines:
        Paths.append(i[0])
        
    Paths = np.array(Paths, dtype = "object")
    return(Paths)

In [47]:

#lap = tf.compat.v1.distributions.Laplace(0.0,1.0)
"""
## Create a sampling layer
"""
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""
    def __init__(self, gamma = 1, **kwargs):
        super(Sampling, self).__init__(**kwargs)
        self.gamma = gamma

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon * self.gamma


In [48]:
def ZMCC(Image1, Image2):
    sd1 = tf.math.reduce_std(Image1, axis = (1,2))
    mean1 = tf.math.reduce_mean(Image1, axis = (1,2), keepdims = True)
    
    sd2 = tf.math.reduce_std(Image2, axis = (1,2))
    mean2 = tf.math.reduce_mean(Image2, axis = (1,2), keepdims = True)

    img1 = (Image1 - mean1)
    img2 = (Image2 - mean2)
    img = img1*img2

    zmcc = (1 - (1 / (128 * 128 * sd1 * sd2)) *  tf.reduce_sum(img, axis=(1,2)))
    return(zmcc)

In [49]:
"""
## Build the encoder
"""

class Encoder(Model):
    def __init__(self, gamma = 0, Size_Kernals=8, Num_Kernals=16, latent_dim = 16, dense_param = 1500000, **kwargs):
        super(Encoder, self).__init__(**kwargs)

        self.Conv1 = layers.Conv2D(Num_Kernals, kernel_size = (Size_Kernals, Size_Kernals), activation="relu", strides=2, padding="same")
        self.Conv2 = layers.Conv2D(Num_Kernals, kernel_size = (Size_Kernals, Size_Kernals), activation="relu", strides=2, padding="same")

        self.flat = layers.Flatten()

        self.DenseParam_Encode = dense_param
        self.DenseNeurons_Encode = np.ceil(self.DenseParam_Encode / (16384.0+2*latent_dim))

        self.dense = layers.Dense(self.DenseNeurons_Encode, activation="relu", kernel_regularizer = l2(0.1))
        self.z_mean = layers.Dense(latent_dim, name="z_mean")
        self.z_log_var = layers.Dense(latent_dim, name="z_log_var", kernel_initializer='zeros', bias_initializer='zeros')
        self.sampling = Sampling(gamma=gamma)
    
    def call(self, inputs):

        x = self.Conv1(inputs)
        x = self.Conv2(x)
        x = self.flat(x)
        x = self.dense(x)
        z_mean = self.z_mean(x)
        z_log_var = self.z_log_var(x)
        z = self.sampling([z_mean, z_log_var])
        return z_mean, z_log_var, z
    
encoder = Encoder(gamma = 0, name="encoder")
encoder(Input(batch_shape=(None,128,128,1)))

encoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_52 (Conv2D)           (None, 64, 64, 16)        1040      
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 32, 32, 16)        16400     
_________________________________________________________________
flatten_26 (Flatten)         (None, 16384)             0         
_________________________________________________________________
dense_31 (Dense)             (None, 92)                1507420   
_________________________________________________________________
z_mean (Dense)               (None, 16)                1488      
_________________________________________________________________
z_log_var (Dense)            (None, 16)                1488      
_________________________________________________________________
sampling_24 (Sampling)       (None, 16)                0   

In [50]:
"""
## Build the decoder
"""


class Decoder(Model):
    def __init__(self, encoder_layer, Size_Kernals=8, Num_Kernals=16, latent_dim=16, dense_param = 1500000, **kwargs):
        super(Decoder, self).__init__(**kwargs)
        Dense_Size = encoder_layer[1]
        
        DenseParam_Decode = dense_param
        Dense_Depth = int(DenseParam_Decode / (latent_dim * Dense_Size * Dense_Size))
        
        self.dense1 = layers.Dense(Dense_Size * Dense_Size * Dense_Depth, activation="relu",  kernel_regularizer = l2(0.1))
        self.dense2 = layers.Reshape((Dense_Size, Dense_Size, Dense_Depth))
                
        self.convT1 = layers.Conv2DTranspose(Num_Kernals, kernel_size = (Size_Kernals, Size_Kernals), activation="relu", strides=2, padding="same")
        self.convT2 = layers.Conv2DTranspose(Num_Kernals, kernel_size = (Size_Kernals, Size_Kernals), activation="relu", strides=2, padding="same")

        self.outputs = layers.Conv2DTranspose(1, kernel_size = (2, 2), activation="relu", padding= "same")
    
    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
                
        x = self.convT1(x)
        x = self.convT2(x)
        
        output = self.outputs(x)
        
        return output
    
decoder = Decoder(encoder.layers[1].output_shape, name="decoder")
decoder(Input(batch_shape=(None, latent_dim)))
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_32 (Dense)             (None, 93184)             1584128   
_________________________________________________________________
reshape_7 (Reshape)          (None, 32, 32, 91)        0         
_________________________________________________________________
conv2d_transpose_21 (Conv2DT (None, 64, 64, 16)        93200     
_________________________________________________________________
conv2d_transpose_22 (Conv2DT (None, 128, 128, 16)      16400     
_________________________________________________________________
conv2d_transpose_23 (Conv2DT (None, 128, 128, 1)       65        
Total params: 1,693,793
Trainable params: 1,693,793
Non-trainable params: 0
_________________________________________________________________


In [51]:
class VAE(Model):
    def __init__(self, encoder, decoder, ZMCC_factor, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = metrics.Mean(name="kl_loss")
        self.ZMCC_factor = ZMCC_factor

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker
        ]

    def train_step(self, data):
        x, y = data
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(x)
            reconstruction = self.decoder(z)
            reconstruction_loss= self.ZMCC_factor * tf.reduce_mean(ZMCC(reconstruction, y))
            beta = 1
            kl_loss = (-0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))) * beta
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)

        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result()
        }

    def call(self, data):
        return self.decoder(self.encoder(data)[2])

#losses.MSE(y, reconstruction), axis=(1, 2)
#losses.mean_squared_logarithmic_error(y, reconstruction), axis=(1, 2)

In [54]:
def train_vae(batch_size=32, epochs=1500, patience=20, ZMCC_factor = 10000, kernal_size=8, kernal_num=16, latent_dim=16, dense_param=1500000):
    
    encoder = Encoder(gamma = 0, 
                      name="encoder", 
                      Size_Kernals=kernal_size, 
                      Num_Kernals=kernal_num,
                      latent_dim=latent_dim,
                      dense_param=dense_param)
    encoder(Input(batch_shape=(None,128,128,1)))
    
    decoder = Decoder(encoder.layers[1].output_shape, 
                      name="decoder", 
                      Size_Kernals=kernal_size, 
                      Num_Kernals=kernal_num,
                      latent_dim=latent_dim,
                      dense_param=dense_param)
    decoder(Input(batch_shape=(None, latent_dim)))
    
    vae = VAE(encoder, decoder, ZMCC_factor)
    vae.compile(optimizer=optimizers.Adam())
    
    

    data_path = "/home/ug-ml/felix-ML/VAE_000/DataAllInOne_Normalised/VAE_000_2/FilePaths/"

    TrainingPathsInput = gen_paths_fromfile(data_path + "TrainingInput_0point1.txt")
    TrainingPathsOutput = gen_paths_fromfile(data_path + "TrainingOutput_0point1.txt")

    ValidationPathsInput = gen_paths_fromfile(data_path + "ValidationInput_0point1.txt")
    ValidationPathsOutput = gen_paths_fromfile(data_path + "ValidationOutput_0point1.txt")

    TestPathsInput = gen_paths_fromfile(data_path + "TestInput_0point1.txt")
    TestPathsOutput = gen_paths_fromfile(data_path + "TestOutput_0point1.txt")

    train_seq = FelixSequence(TrainingPathsInput, TrainingPathsOutput, batch_size)
    val_seq = FelixSequence(ValidationPathsInput, ValidationPathsOutput, batch_size)
    test_seq = FelixSequence(TestPathsInput, TestPathsOutput, batch_size)

    best_model_name = "VAE_000_Normalised_0point1_zmcc10000"
    
    best_model = vae


    patience_i = 0
    best_val_loss = np.inf

    #training and validation histories, containing [0] the total loss, [1] the reconstruction loss, and [2] the kl loss.
    #val_hist = np.zeros(shape=(1,epochs))
    #train_hist = np.zeros(shape=(3,epochs))

    for epoch in range(0, epochs):
        print("-------------------------------------------------------------------------")
        print("Epoch", epoch, "/", epochs, ": ")
        print("Training: ")
        vae.encoder.sampling.gamma=1
        hist = vae.fit(x = train_seq, shuffle=True, epochs = epoch+1, workers = 16, initial_epoch=epoch)
        #train_hist[0][epoch] = hist.history["loss"][0]
        #train_hist[1][epoch] = hist.history["reconstruction_loss"][0]
        #train_hist[2][epoch] = hist.history["kl_loss"][0]
        print("Validation: ")

        tot_batch_recon_loss = 0
        count = 0
        vae.encoder.sampling.gamma=0
        for x, y in val_seq:
            count += 1
            reconstruction = vae(x)
            reconstruction_loss= ZMCC_factor * tf.reduce_mean(ZMCC(reconstruction, y))

            tot_batch_recon_loss += reconstruction_loss

        avg_recon_loss = float(tot_batch_recon_loss/count)
        if(avg_recon_loss < best_val_loss):
            #vae.save("/home/ug-ml/felix-ML/VAE_000/DataAllInOne_Normalised/VAE_000_2/Models/"+str(best_model_name))
            best_model = vae
            print("The model improved from: ",best_val_loss, "to: ", avg_recon_loss)
            best_val_loss = avg_recon_loss
            patience_i = 0
        else:
            patience_i+=1
            print("The model did not improve, patience_i = ", patience_i)

        print("Average reconstruction loss: ", avg_recon_loss)
        #val_hist[0][epoch] = avg_recon_loss
        if(patience_i > patience):
            print("Early Stopping, the model did not improve from: ", best_val_loss)
            break

    print("-------------------------------------------------------------------------")
    
    test_recon_loss = 0
    count = 0
    best_model.encoder.sampling.gamma=0
    for x, y in test_seq:
        count += 1
        reconstruction = best_model(x)
        reconstruction_loss= ZMCC_factor * tf.reduce_mean(ZMCC(reconstruction, y))

        test_recon_loss += reconstruction_loss

    avg_recon_loss = float(test_recon_loss/count)
    
    return avg_recon_loss, best_model
    
    




In [55]:
parameters = {"batch_size": np.array([8, 12, 16, 24, 32, 48, 64, 86, 128]),
             "kernal_size": np.array([2, 4, 6, 8, 10, 12, 14, 16, 18]),
             "kernal_num": np.array([8, 16, 24, 32, 40, 48, 56, 64, 72]),
             "ZMCC_factor": np.array([10, 50, 100, 500, 1000, 5000, 10000, 50000, 100000]),
             "latent_dim": np.array([2,4,6,8,10,12,14,16,18]),
             "dense_param": np.array([500000, 750000, 1000000, 1250000, 1500000, 1750000, 2000000, 2250000, 2500000])}
    
z = np.array([4, 3, 3, 6, 5, 4])
prev_searched = np.array([[4, 3, 3, 6, 5, 4]])

epochs = 1500

def neighbours(point, dircs):
    ns = dircs+point
    #print(dircs, point, prev_searched)
    return np.array([i for i in ns if (0<=i).all() and (i<9).all() and not (i == prev_searched).all(axis=1).any()]).astype(int)

def parameter_search(parameters, z, prev_searched):
    
    num_params = len(parameters)
    dircs = np.zeros(shape = (2*num_params, num_params))
    for i, dirc in enumerate(dircs):
        if i < num_params:
            dirc[i] = 1
        else:
            dirc[i-num_params] = -1
    
    best_params = {}
    for key in parameters:
        best_params[key] = np.nan

    best_metrics = {"test_loss": np.inf, "test_acc": -np.inf}

    best_hist = {"train_acc": np.zeros(shape=epochs),
                "train_loss": np.zeros(shape=epochs),
                "val_acc": np.zeros(shape=epochs),
                "val_loss": np.zeros(shape=epochs)}
    
    converged = False
        
    while not converged:
        print("ITERATING OVER:")
        neighs = neighbours(z, dircs)
        num_neighs = neighs.shape[0]
        print(neighs, num_neighs)

        if neighs.size == 0:
            print("No new neighbours available. Saving best model and parameter set so far.")
            #best_model.save(NewPath+ModelName)
            converged = True
            break

        step_params = np.array([parameters[key][neighs[:,j]] for j, key in enumerate(parameters)]).T
        #print(step_params)
                

        converged = True

        for i, param_set in enumerate(step_params):
            print("Parameter set ", i+1," of ", num_neighs,"+++++++++++++++++++++++++++++++++++++++++++++++++++.")
            
            loss_norm_factor = 10000/param_set[3]
            
            test_loss, best_model = train_vae(batch_size=param_set[0],
                                             epochs=epochs,
                                             ZMCC_factor=param_set[3], 
                                             kernal_size=param_set[1], 
                                             kernal_num=param_set[2],
                                             latent_dim=param_set[4],
                                             dense_param=param_set[5])
            
            test_loss = test_loss*loss_norm_factor

            prev_searched = np.append(prev_searched, neighs[i].reshape(1,num_params), axis=0)

            if test_loss < best_metrics["test_loss"]:
                
                best_metrics["test_loss"] = test_loss
                
                for k, key in enumerate(best_params):
                    best_params[key] = param_set[k]

                z = neighs[i]
                
                #best_model.save(NewPath+ModelName)
                converged = False
        print("best params set:" )
        print(best_params)
        print("best metrics set:" )
        print(best_metrics)
                
parameter_search(parameters, z, prev_searched)

ITERATING OVER:
[[5 3 3 6 5 4]
 [4 4 3 6 5 4]
 [4 3 4 6 5 4]
 [4 3 3 7 5 4]
 [4 3 3 6 6 4]
 [4 3 3 6 5 5]
 [3 3 3 6 5 4]
 [4 2 3 6 5 4]
 [4 3 2 6 5 4]
 [4 3 3 5 5 4]
 [4 3 3 6 4 4]
 [4 3 3 6 5 3]] 12
Parameter set  1  of  12 +++++++++++++++++++++++++++++++++++++++++++++++++++.
-------------------------------------------------------------------------
Epoch 0 / 1500 : 
Training: 
93/93 [==============================] - 4s 44ms/step - loss: 6727.6079 - reconstruction_loss: 5703.0986 - kl_loss: 12.6305
Validation: 
The model improved from:  inf to:  4732.63232421875
Average reconstruction loss:  4732.63232421875
-------------------------------------------------------------------------
Epoch 1 / 1500 : 
Training: 
Epoch 2/2
93/93 [==============================] - 4s 43ms/step - loss: 4494.4842 - reconstruction_loss: 4323.6421 - kl_loss: 42.8148
Validation: 
The model improved from:  4732.63232421875 to:  4071.238525390625
Average reconstruction loss:  4071.238525390625
-------------------

Validation: 
The model did not improve, patience_i =  1
Average reconstruction loss:  1761.0010986328125
-------------------------------------------------------------------------
Epoch 22 / 1500 : 
Training: 
Epoch 23/23
93/93 [==============================] - 5s 58ms/step - loss: 1219.1617 - reconstruction_loss: 1170.9476 - kl_loss: 48.2397
Validation: 
The model improved from:  1758.5206298828125 to:  1727.524169921875
Average reconstruction loss:  1727.524169921875
-------------------------------------------------------------------------
Epoch 23 / 1500 : 
Training: 
Epoch 24/24
93/93 [==============================] - 5s 55ms/step - loss: 1189.8829 - reconstruction_loss: 1108.9836 - kl_loss: 44.3773
Validation: 
The model improved from:  1727.524169921875 to:  1649.557861328125
Average reconstruction loss:  1649.557861328125
-------------------------------------------------------------------------
Epoch 24 / 1500 : 
Training: 
Epoch 25/25
93/93 [==============================] - 4

93/93 [==============================] - 4s 48ms/step - loss: 752.5484 - reconstruction_loss: 709.3431 - kl_loss: 45.8774
Validation: 
The model improved from:  1427.1522216796875 to:  1409.0928955078125
Average reconstruction loss:  1409.0928955078125
-------------------------------------------------------------------------
Epoch 46 / 1500 : 
Training: 
Epoch 47/47
93/93 [==============================] - 6s 70ms/step - loss: 668.0038 - reconstruction_loss: 634.6108 - kl_loss: 45.1450 0s - loss: 665.9217 - reconstruction_loss: 642.2108 - 
Validation: 
The model improved from:  1409.0928955078125 to:  1373.617919921875
Average reconstruction loss:  1373.617919921875
-------------------------------------------------------------------------
Epoch 47 / 1500 : 
Training: 
Epoch 48/48
93/93 [==============================] - 6s 66ms/step - loss: 641.7260 - reconstruction_loss: 623.9628 - kl_loss: 45.3359
Validation: 
The model did not improve, patience_i =  1
Average reconstruction loss:  1

93/93 [==============================] - 4s 43ms/step - loss: 480.8028 - reconstruction_loss: 433.0528 - kl_loss: 45.0275
Validation: 
The model improved from:  1292.9024658203125 to:  1274.2054443359375
Average reconstruction loss:  1274.2054443359375
-------------------------------------------------------------------------
Epoch 69 / 1500 : 
Training: 
Epoch 70/70
93/93 [==============================] - 5s 49ms/step - loss: 455.0450 - reconstruction_loss: 431.9055 - kl_loss: 44.4722
Validation: 
The model did not improve, patience_i =  1
Average reconstruction loss:  1392.03076171875
-------------------------------------------------------------------------
Epoch 70 / 1500 : 
Training: 
Epoch 71/71
93/93 [==============================] - 5s 57ms/step - loss: 538.8017 - reconstruction_loss: 547.9538 - kl_loss: 45.8532
Validation: 
The model did not improve, patience_i =  2
Average reconstruction loss:  1411.6927490234375
---------------------------------------------------------------

93/93 [==============================] - 4s 45ms/step - loss: 430.3711 - reconstruction_loss: 368.2713 - kl_loss: 44.5444
Validation: 
The model did not improve, patience_i =  4
Average reconstruction loss:  1287.75830078125
-------------------------------------------------------------------------
Epoch 92 / 1500 : 
Training: 
Epoch 93/93
93/93 [==============================] - 7s 73ms/step - loss: 406.9364 - reconstruction_loss: 364.0152 - kl_loss: 44.5461
Validation: 
The model did not improve, patience_i =  5
Average reconstruction loss:  1265.6319580078125
-------------------------------------------------------------------------
Epoch 93 / 1500 : 
Training: 
Epoch 94/94
93/93 [==============================] - 6s 66ms/step - loss: 389.0121 - reconstruction_loss: 355.9433 - kl_loss: 44.0947
Validation: 
The model did not improve, patience_i =  6
Average reconstruction loss:  1305.25634765625
-------------------------------------------------------------------------
Epoch 94 / 1500 :

...

186/186 [==============================] - 9s 50ms/step - loss: 427.0106 - reconstruction_loss: 381.5213 - kl_loss: 43.9719
Validation: 
The model did not improve, patience_i =  2
Average reconstruction loss:  1310.33740234375
-------------------------------------------------------------------------
Epoch 76 / 1500 : 
Training: 
Epoch 77/77
186/186 [==============================] - 7s 37ms/step - loss: 439.0110 - reconstruction_loss: 420.8311 - kl_loss: 44.6682
Validation: 
The model did not improve, patience_i =  3
Average reconstruction loss:  1390.9793701171875
-------------------------------------------------------------------------
Epoch 77 / 1500 : 
Training: 
Epoch 78/78
186/186 [==============================] - 6s 32ms/step - loss: 488.7312 - reconstruction_loss: 458.4818 - kl_loss: 45.1687
Validation: 
The model did not improve, patience_i =  4
Average reconstruction loss:  1292.991943359375
-------------------------------------------------------------------------
Epoch 78 /

186/186 [==============================] - 7s 37ms/step - loss: 391.8291 - reconstruction_loss: 338.1290 - kl_loss: 44.0772 1s - loss: 395.3738 - reconstruction_loss: 344.0040 - kl_loss: 4 - - ETA: 0s - loss: 392.5236 - reconstruction_loss: 340.7201 
Validation: 
The model did not improve, patience_i =  13
Average reconstruction loss:  1236.1080322265625
-------------------------------------------------------------------------
Epoch 100 / 1500 : 
Training: 
Epoch 101/101
186/186 [==============================] - 5s 26ms/step - loss: 386.8316 - reconstruction_loss: 354.1078 - kl_loss: 43.8607
Validation: 
The model did not improve, patience_i =  14
Average reconstruction loss:  1285.4986572265625
-------------------------------------------------------------------------
Epoch 101 / 1500 : 
Training: 
Epoch 102/102
186/186 [==============================] - 5s 27ms/step - loss: 479.8825 - reconstruction_loss: 400.8630 - kl_loss: 44.3594
Validation: 
The model did not improve, patience_i 

186/186 [==============================] - 7s 40ms/step - loss: 321.6210 - reconstruction_loss: 281.2972 - kl_loss: 43.6197
Validation: 
The model improved from:  1212.5545654296875 to:  1209.467041015625
Average reconstruction loss:  1209.467041015625
-------------------------------------------------------------------------
Epoch 123 / 1500 : 
Training: 
Epoch 124/124
186/186 [==============================] - 7s 35ms/step - loss: 303.4483 - reconstruction_loss: 273.3076 - kl_loss: 43.6729
Validation: 
The model improved from:  1209.467041015625 to:  1207.382080078125
Average reconstruction loss:  1207.382080078125
-------------------------------------------------------------------------
Epoch 124 / 1500 : 
Training: 
Epoch 125/125
186/186 [==============================] - 9s 48ms/step - loss: 330.9099 - reconstruction_loss: 288.1682 - kl_loss: 43.4787
Validation: 
The model improved from:  1207.382080078125 to:  1202.8048095703125
Average reconstruction loss:  1202.8048095703125
---

186/186 [==============================] - 8s 43ms/step - loss: 326.1547 - reconstruction_loss: 281.2097 - kl_loss: 43.7480
Validation: 
The model did not improve, patience_i =  7
Average reconstruction loss:  1228.1934814453125
-------------------------------------------------------------------------
Epoch 146 / 1500 : 
Training: 
Epoch 147/147
186/186 [==============================] - 9s 49ms/step - loss: 317.8195 - reconstruction_loss: 259.1622 - kl_loss: 43.4218
Validation: 
The model did not improve, patience_i =  8
Average reconstruction loss:  1199.74365234375
-------------------------------------------------------------------------
Epoch 147 / 1500 : 
Training: 
Epoch 148/148
186/186 [==============================] - 5s 26ms/step - loss: 294.6303 - reconstruction_loss: 261.0366 - kl_loss: 43.4887
Validation: 
The model did not improve, patience_i =  9
Average reconstruction loss:  1224.6951904296875
-------------------------------------------------------------------------
Epo

186/186 [==============================] - 5s 28ms/step - loss: 285.6544 - reconstruction_loss: 234.7433 - kl_loss: 43.0374
Validation: 
The model did not improve, patience_i =  3
Average reconstruction loss:  1215.3184814453125
-------------------------------------------------------------------------
Epoch 169 / 1500 : 
Training: 
Epoch 170/170
186/186 [==============================] - 8s 41ms/step - loss: 294.5276 - reconstruction_loss: 261.2573 - kl_loss: 43.5354
Validation: 
The model did not improve, patience_i =  4
Average reconstruction loss:  1241.5277099609375
-------------------------------------------------------------------------
Epoch 170 / 1500 : 
Training: 
Epoch 171/171
186/186 [==============================] - 6s 35ms/step - loss: 308.2010 - reconstruction_loss: 257.5410 - kl_loss: 43.3945
Validation: 
The model did not improve, patience_i =  5
Average reconstruction loss:  1192.2088623046875
-------------------------------------------------------------------------
E

In [ ]:
print()

In [110]:
prev_searched = np.array([[0,1,2]])

learning_rate = np.array([0.0005, 0.0001, 0.001, 0.002, 0.005])
l2_regularizer = np.array([0.00005, 0.0001, 0.0005, 0.001, 0.002])
batch_size = np.array([8, 16, 32, 64, 128])

def neighbours(point):
    dircs = np.array([[0,0,1],[0,1,0],[1,0,0],[0,0,-1],[0,-1,0],[-1,0,0]])
    ns = dircs+point
    return np.array([i for i in ns if (0<=i).all() and (i<5).all() and not (i == prev_searched).all(axis=1).any()])


z = np.array([0,1,2])
converged = False
best_test_loss = np.inf
best_lr = np.nan
best_l2_r = np.nan
best_bs = np.nan

while not converged:
    print("ITERATING OVER:")
    neighs = neighbours(z)
    print(neighs)
    #print(neighs)
    if neighs.size == 0:
        print("No new neighbours available. Saving best model and parameter set so far.")
        np.save(NewPath+"/parameter_search.npy", np.array(best_lr,best_l2_r,best_bs))
        best_model.save(NewPath+ModelName)
        converged = True
        break
        

    lr = learning_rate[neighs[:,0]]
    l2_r = l2_regularizer[neighs[:,1]]
    bs = batch_size[neighs[:,2]]

    step_params = np.array([lr, l2_r, bs]).T
    
    converged = True
    
    for i, param_set in enumerate(step_params):
        
        model = build_model(param_set[0], param_set[1])
        
        test_loss, best_model = felix_fit_new(model, param_set[2].astype(int), epochs, CPUworkers, AllPaths, "npy", TrainingPatience)
        
        prev_searched = np.append(prev_searched, neighs[i].reshape(1,3), axis=0)
        
        if test_loss < best_test_loss:
            best_test_loss = test_loss
            best_lr = param_set[0]
            best_l2_r = param_set[1]
            best_bs = param_set[2]
            
            z = neighs[i]
            
            np.save(NewPath+"/parameter_search.npy",param_set)
            best_model.save(NewPath+ModelName)
            converged = False
            
    

ITERATING OVER:
[[0 1 3]
 [0 2 2]
 [1 1 2]
 [0 1 1]
 [0 0 2]]
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
-------------------------------------------------------------------------
Epoch 1 / 1 : 
Training: 
89/89 [==============================] - 7s 83ms/step - loss: 2.0421 - acc: 0.2734
Validation: 
11/11 [==============================] - 0s 29ms/step - loss: 2.5310 - acc: 0.3549
The model improved from:  inf to:  2.5309948921203613
Epoch loss:  2.5309948921203613
-------------------------------------------------------------------------
Testing: 
6/6 [==============================] - 0s 20ms/step - loss: 1.9448 - acc: 0.3724
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
-------------------------------------------------------------------------
Epoch 1 / 1 : 
Training: 
177/177 [=

In [112]:
print(best_lr,best_l2_r,best_bs)
print(best_test_loss)

0.0005 0.0005 32.0
1.822420358657837


In [38]:
test_seq = FelixSequence(AllPaths[2][0], AllPaths[2][1], best_bs.astype(int), "npy")
tst_hist = best_model.evaluate(test_seq, workers = 16)

11/11 [==============================] - 0s 10ms/step - loss: 1.6834 - acc: 0.6216


In [ ]:
#Build model
strategy = MirroredStrategy() #Allows multiple GPUs

with strategy.scope():
    model = models.Sequential()
    model.add(layers.Conv2D(128, (4, 4),
                                     activation='relu',
                                     data_format='channels_first',
                                     input_shape= input_shape))
    model.add(layers.MaxPooling2D((2, 2), data_format='channels_first'))
    model.add(layers.Conv2D(128, (4, 4),
                                     data_format='channels_first',
                                     activation='relu'))
    model.add(layers.MaxPooling2D((2, 2), data_format='channels_first'))
    model.add(layers.Conv2D(128, (4, 4),
                                     data_format='channels_first',
                                     activation='relu'))
    model.add(layers.MaxPooling2D((2, 2), data_format='channels_first'))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(128, activation='relu',
                           kernel_regularizer = l2(l2_regularizer)))
    
    model.add(layers.Dense(10, activation='softmax',
                           kernel_regularizer = l2(l2_regularizer)))

    model.compile(loss = loss,
                  optimizer = optimizers.RMSprop(learning_rate = learning_rate),
                  metrics=['acc'])
    
#Make folder to put model and history information
try:
    os.mkdir(NewPath)
except:
    print("Folder failed to be created, it may already exist")
    
File1  = open(NewPath +"/Parameters.txt", "w+")
if(len(VariableListName) == len(VariableListValues)):
    for i in range(0, len(VariableListName)):
        File1.write(VariableListName[i] + " " + str(VariableListValues[i]) + "\n")
    File1.close()
else:
    print("VariableListName and VariableListValues do not match up, so file can not be saved")

In [ ]:
training_history = np.zeros(shape=(2,epochs))
validation_history = np.zeros(shape=(2,epochs))
test_history = [0,0]
#print(model.metrics_names)
felix_fit_new(model, batch_size, epochs, CPUworkers, AllPaths, "npy",TrainingPatience)